In [84]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ranked-board-game-data-from-boardgamegeek/mechanisms_2023.csv
/kaggle/input/ranked-board-game-data-from-boardgamegeek/reimplementations_2023.csv
/kaggle/input/ranked-board-game-data-from-boardgamegeek/themes_2023.csv
/kaggle/input/ranked-board-game-data-from-boardgamegeek/subdomains_2023.csv
/kaggle/input/ranked-board-game-data-from-boardgamegeek/basic_data_2023.csv


In [121]:
# similar names
from fuzzywuzzy import process


In [165]:
# randomize akinator questions
from random import randint

# Reading datasets

In [85]:
df_games = pd.read_csv("/kaggle/input/ranked-board-game-data-from-boardgamegeek/basic_data_2023.csv", 
                       index_col='game_id')
upper_limit_id_games = df_games.index.max()
min_limit_id_games = df_games.index.min()
df_games.columns

Index(['rank', 'bgg_url', 'name', 'min_players', 'max_players', 'avg_time',
       'min_time', 'max_time', 'weight', 'year', 'age', 'avg_rating',
       'geek_rating', 'num_votes', 'owned', 'designer'],
      dtype='object')

In [86]:
df_mechs = pd.read_csv("/kaggle/input/ranked-board-game-data-from-boardgamegeek/mechanisms_2023.csv",
                      index_col = 'game_id')
df_mechs.columns

Index(['Acting', 'Action Drafting', 'Action Points', 'Action Queue',
       'Action Retrieval', 'Action Timer', 'Action/Event', 'Advantage Token',
       'Alliances', 'Area Majority / Influence',
       ...
       'Turn Order: Time Track', 'Variable Phase Order',
       'Variable Player Powers', 'Variable Set-up',
       'Victory Points as a Resource', 'Voting', 'Worker Placement',
       'Worker Placement with Dice Workers',
       'Worker Placement, Different Worker Types', 'Zone of Control'],
      dtype='object', length=188)

In [87]:
df_themes = pd.read_csv('/kaggle/input/ranked-board-game-data-from-boardgamegeek/themes_2023.csv',
                       index_col = 'game_id')


In [88]:
df_subdomains = pd.read_csv("/kaggle/input/ranked-board-game-data-from-boardgamegeek/subdomains_2023.csv",
                           index_col = 'game_id')


# Usando a função de obter resultados através do Índice

In [89]:
continuing = 'S'
while continuing == 'S':
    
    #Structure to ensure game_id does exist
    wanted_id = upper_limit_id_games+1
    while df_games['name'].get(wanted_id) == None:
        wanted_id = int(input(f"Digite o código do jogo que você quer pesquisar, entre {min_limit_id_games} e {upper_limit_id_games}): "))
        while (wanted_id > upper_limit_id_games or wanted_id < min_limit_id_games):
            wanted_id = int(input(f"Digite o código do jogo que você quer pesquisar, entre {min_limit_id_games} e {upper_limit_id_games}): "))
        wanted_name = df_games['name'].get(wanted_id)
        if wanted_name == None:
            print("Não existe código correspondente a este jogo...")
            wanted_sup = wanted_id
            wanted_inf = wanted_id
            while df_games['name'].get(wanted_sup) == None:
                wanted_sup = wanted_sup + 1

            while df_games['name'].get(wanted_inf) == None:
                wanted_inf = wanted_inf - 1


            print(f"Os números mais próximos são: {wanted_sup} ou {wanted_inf}") 
        else:

            break

    print(f"O jogo com o código {wanted_id} é o {wanted_name}.")

    # Select the row from the index
    wanted_mechs = df_mechs.loc[wanted_id]
    wanted_themes = df_themes.loc[wanted_id]
    wanted_subdomains = df_subdomains.loc[wanted_id]

    # Obtain the nemas of columns whete from the index
    wanted_mechs = wanted_mechs.index[wanted_mechs == 1].tolist()
    wanted_themes = wanted_themes.index[wanted_themes == 1].tolist()
    wanted_subdomains = wanted_subdomains.index[wanted_subdomains == 1].tolist()

#    print(f"as mecânicas deste jogo são: {wanted_mechs} \nos temas do jogo são: {wanted_themes} \nos subdomínios do jogo são: {wanted_subdomains}")

    continuing = str.upper(input("Gostaria de selecionar outro? [S/N]"))
    if continuing == 'N':
        break

Digite o código do jogo que você quer pesquisar, entre 1 e 374173):  200000


Não existe código correspondente a este jogo...
Os números mais próximos são: 200057 ou 199966


Digite o código do jogo que você quer pesquisar, entre 1 e 374173):  200057


O jogo com o código 200057 é o Pioneer Days.


Gostaria de selecionar outro? [S/N] s
Digite o código do jogo que você quer pesquisar, entre 1 e 374173):  199966


O jogo com o código 199966 é o Kingsburg (Second Edition).


Gostaria de selecionar outro? [S/N] n


In [90]:
df_similar_mechs = df_mechs[wanted_mechs]
df_similar_mechs['sum'] = df_similar_mechs.sum(axis=1)

#list of 2 top sums
top2_sums_mechs = df_similar_mechs['sum'].unique().tolist()
top2_sums_mechs.sort(reverse=True)
top2_sums_mechs = top2_sums_mechs[0:2]

#exclude 0'es in list 
if 0 in top2_sums_mechs:
    top2_sums_mechs.remove(0)

#games with 2 top sums    
only_most_similar_games_mechs = df_similar_mechs.loc[df_similar_mechs['sum'].isin(top2_sums_mechs)].index.tolist()

# Remove os elementos iguais ao valor especificado
only_most_similar_games_mechs.remove(wanted_id)

only_most_similar_games_mechs = df_games.name.loc[only_most_similar_games_mechs].tolist()


/tmp/ipykernel_42/1514433514.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_similar_mechs['sum'] = df_similar_mechs.sum(axis=1)


In [91]:
df_similar_themes = df_themes[wanted_themes]
df_similar_themes['sum'] = df_similar_themes.sum(axis=1)


#list of 2 top sums
top2_sums_themes = df_similar_themes['sum'].unique().tolist()
top2_sums_themes.sort(reverse=True)
top2_sums_themes = top2_sums_themes[0:2]

#exclude 0'es in list 
if 0 in top2_sums_themes:
    top2_sums_themes.remove(0)

#games with 2 top sums    
only_most_similar_games_themes = df_similar_themes.loc[df_similar_themes['sum'].isin(top2_sums_themes)].index.tolist()

# Remove os elementos iguais ao valor especificado
only_most_similar_games_themes.remove(wanted_id)

only_most_similar_games_themes = df_games.name.loc[only_most_similar_games_themes].tolist()

/tmp/ipykernel_42/2352962090.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_similar_themes['sum'] = df_similar_themes.sum(axis=1)


In [119]:
print(15*'-*-')
print(f"O jogo com o código {wanted_id} é o {wanted_name}. \n")
print(15*'-*-')
print(f"As mecânicas deste jogo são:") 
for item in range(len(wanted_mechs)):
    print(f"  - {wanted_mechs[item]}")
print(f'Os jogos com mecânicas mais similares são: ')
for item in range(len(only_most_similar_games_mechs)):
    print(f'  - {only_most_similar_games_mechs[item]} - ano ', end = "")
    print(df_games.loc[df_games.name == only_most_similar_games_mechs[item],'year'].iloc[0], end = "")
    print(" - peso ", end = "")
    print(df_games.loc[df_games['name'] == only_most_similar_games_mechs[item],'weight'].iloc[0])
print(15*'-*-')
print(f"Os temas deste jogo são:")
for item in range(len(wanted_themes)):
    print(f"  - {wanted_themes[item]}") 
print(f'Os jogos com temas mais similares são: ')
for item in range(len(only_most_similar_games_themes)):
    print(f'  - {only_most_similar_games_themes[item]} - ano ', end = "")
    print(df_games.loc[df_games.name == only_most_similar_games_themes[item],'year'].iloc[0], end = "")
    print(" - peso ", end = "")
    print(df_games.loc[df_games['name'] == only_most_similar_games_themes[item],'weight'].iloc[0])
print(15*'-*-')
print(f"Os subdomínios do jogo são: {wanted_subdomains}")


-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
O jogo com o código 199966 é o Kingsburg (Second Edition). 

-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
As mecânicas deste jogo são:
  - Dice Rolling
  - Tech Trees / Tech Tracks
  - Worker Placement
  - Worker Placement with Dice Workers
Os jogos com mecânicas mais similares são: 
  - The Castles of Burgundy - ano 2011 - peso 2.9914
  - The Voyages of Marco Polo - ano 2015 - peso 3.1879
  - Troyes - ano 2010 - peso 3.4607
  - Lorenzo il Magnifico - ano 2016 - peso 3.3002
  - Roll for the Galaxy - ano 2014 - peso 2.7759
  - Rajas of the Ganges - ano 2017 - peso 2.8975
  - Marco Polo II: In the Service of the Khan - ano 2019 - peso 3.3793
  - Coimbra - ano 2018 - peso 3.2489
  - Under Falling Skies - ano 2020 - peso 2.3965
  - The Taverns of Tiefenthal - ano 2019 - peso 2.6869
  - Pulsar 2849 - ano 2017 - peso 3.3444
  - Bora Bora - ano 2013 - peso 3.7417
  - Alien Frontiers - ano 2010 - peso 2.5686
  - Kingsburg - ano 2007 - peso 2.4227


# Agora um novo teste de nomes similares

In [160]:
option = 0
while option == 0:
    game = input("Which game are you looking for?")
    all_games = df_games['name']

#compare names
    results = process.extract(game,all_games)
    print(f"the names similars to {game} are: ")
    for option in range(len(results)):
        print(f' [{option+1}] - {results[option][0]}')
    option = int(input('Choose an option: "0" to choose again'))
print(15*'-+-')
df_games.loc[df_games.name == results[option-1][0]].T

Which game are you looking for? zombie


the names similars to zombie are: 
 [1] - Zombie Kidz Evolution
 [2] - Last Night on Earth: The Zombie Game
 [3] - Tiny Epic Zombies
 [4] - Zombie Teenz Evolution
 [5] - Zombicide


Choose an option: "0" to choose again 5


-+--+--+--+--+--+--+--+--+--+--+--+--+--+--+-


game_id,113924
rank,618
bgg_url,https://boardgamegeek.com/boardgame/113924
name,Zombicide
min_players,1
max_players,6
avg_time,60
min_time,60
max_time,60
weight,2.5351
year,2012


# Teste Akinator

In [164]:
print(df_games.shape)
df_intermediate = df_games.merge(df_themes, how = 'left', on = df_games.index)
df_total = df_intermediate.merge(df_mechs, how = 'left')
df_total.shape

(2000, 16)


(3389123, 286)

In [ ]:
# Fórmula de comparação
def total_players(valor):
    # Substitua esta função pela sua lógica de comparação
    if df_total.min_players <= players and df_total.max_players >= players:
            df_total['akinator'] = df_total['akinator'].apply(lambda x: x + 10)

    return valor > 10

# Aplica a fórmula à coluna 'akinator'


In [163]:
df_total['akinator'] = 0
# condition to continue the questions
while df_total['akinator'].nunique() != 1 or df_total['akinator'].max() != df_total['akinator'].iloc[0]:
    # Realize as perguntas aqui, atualizando o DataFrame df_total conforme necessário
    players = input("How many players are you looking for?")
        
    # Pode ser uma entrada do usuário ou outro método de atualização
        df_total['comparacao'] = df_total['akinator'].apply(total_players)

    # Atualize os valores na coluna 'akinator' para simular uma mudança nos valores

# Imprima o DataFrame quando a condição for atendida
print("DataFrame final:")
print(df_total)

0          None
1          None
2          None
3          None
4          None
           ... 
3389118    None
3389119    None
3389120    None
3389121    None
3389122    None
Name: sum, Length: 3389123, dtype: object